In [ ]:
reset -f


In [ ]:
import os
import pandas as pd
import numpy as np
import json
import re
from Utils import print_dst


In [ ]:
# start_date: str = "2022-06-14"
# end_date: str = "2022-12-31"
start_date: str = "2023-01-01"
end_date: str = "2023-12-31"
# start_date = pd.to_datetime(start_date, utc=True).tz_convert('Europe/Helsinki')
# end_date = pd.to_datetime(end_date, utc=True).tz_convert('Europe/Helsinki')
start_date = print_dst(start_date)
end_date = print_dst(end_date)


In [ ]:
input_folder: str = 'InputData'
preprocessed_input_folder: str = 'PreprocessedData'
defa_file_name_gzip: str = 'Defa_Table.gzip'
electricity_price_file_name_gzip: str = 'Electricity_Price_Table.gzip'
chargeing_point_owner_changes_file_name: str = "Chargeing_point_owner_changes.json"
chargeing_point_name_file_name: str = "Chargeing_point_name.json"
mittarilukemat_file_name: str = "Mittarilukemat.json"
mittalukemat_2023_01_01_file_name: str = "Mittarilukemat_2023_01_01.csv"


In [ ]:
file_path: str = os.path.join(preprocessed_input_folder, defa_file_name_gzip)
df_defa: pd.DataFrame = pd.read_parquet(file_path)
file_path: str = os.path.join(preprocessed_input_folder, electricity_price_file_name_gzip)
df_electricity_price: pd.DataFrame = pd.read_parquet(file_path)
file_path: str = os.path.join(input_folder, chargeing_point_owner_changes_file_name)
file = open(file_path, "r")
df_chargeing_point_owner_changes: pd.DataFrame = pd.read_json(file, lines=True)
file.close()
file_path: str = os.path.join(input_folder, chargeing_point_name_file_name)
file = open(file_path, "r")
df_chargeing_point_name: pd.DataFrame = pd.read_json(file, lines=True)
file.close()
file_path: str = os.path.join(input_folder, mittarilukemat_file_name)
file = open(file_path, "r")
df_mittarilukemat: pd.DataFrame = pd.read_json(file, lines=True)
file.close()
df_mittarilukemat["DateTime"] = pd.to_datetime(df_mittarilukemat["DateTime"])


In [ ]:
df_mittarilukemat: pd.DataFrame = pd.merge(df_mittarilukemat, df_chargeing_point_name)
df_mittarilukemat.head(20)


In [ ]:
df_chargeing_point_name.head()


In [ ]:
df_chargeing_point_owner_changes: pd.DataFrame = pd.merge(df_chargeing_point_owner_changes, df_chargeing_point_name)
df_chargeing_point_owner_changes["start_date"] = pd.to_datetime(df_chargeing_point_owner_changes["start_date"])
df_chargeing_point_owner_changes["end_date"] = pd.to_datetime(df_chargeing_point_owner_changes["end_date"])


In [ ]:
df_chargeing_point_owner_changes.dtypes


In [ ]:
df_chargeing_point_owner_changes.head(40)


In [ ]:
# df_defa: pd.DataFrame = pd.merge(df_defa, df_chargeing_point_name, left_on=["id", "connector_id"],
#                                  right_on=["id", "connectorId"])
df_defa: pd.DataFrame = pd.merge(df_defa, df_chargeing_point_name, left_on=["id", "connectorId"],
                                 right_on=["id", "connectorId"])
df_defa.head()


In [ ]:
df_defa.dtypes


In [ ]:
df_electricity_price.head()


In [ ]:
df_electricity_price.dtypes


In [ ]:
def add_chargeing_time_column(df: pd.DataFrame, row: pd.Series, start_date_and_hour, end_date_and_hour):
    if start_date_and_hour == end_date_and_hour:
        df['chargeing_time'] = (row['end_time'] - row['start_time']).total_seconds()
    else:
        df.loc[df['DateTime'] == start_date_and_hour, 'chargeing_time'] = \
                                            ((start_date_and_hour - row['start_time']).total_seconds() + 3600)
        df.loc[df['DateTime'] == end_date_and_hour, 'chargeing_time'] = \
                                            (row['end_time'] - end_date_and_hour).total_seconds()


In [ ]:
def set_owner(start_date: str, id: str, connector_id: str) -> str:
    df: pd.DataFrame = \
        df_chargeing_point_owner_changes[(df_chargeing_point_owner_changes["id"] == id) &
                    (df_chargeing_point_owner_changes["connectorId"] == connector_id) &
                    ((df_chargeing_point_owner_changes["start_date"] <= start_date) |
                    df_chargeing_point_owner_changes["start_date"].isna()) &
                    ((df_chargeing_point_owner_changes["end_date"] >= start_date) |
                    df_chargeing_point_owner_changes["end_date"].isna()) ]
    if df.shape[0] > 0:
        if type(df.iloc[0]["start_date"]) == pd._libs.tslibs.nattype.NaTType:
            time_stamp = df.iloc[0]["end_date"]
            owner_prefix: str = "E_"
        else:
            owner_prefix: str = "S_"
            time_stamp = df.iloc[0]["start_date"]
        owner: str = "{}{}".format(owner_prefix, time_stamp.strftime("%Y-%m-%d"))
    else:
        owner: str = "--"
    return owner


In [ ]:
def create_list_of_chargeing_times(row: pd.Series, df_electricity_price: pd.DataFrame) -> pd.DataFrame:
    start_date_and_hour: str = pd.Timestamp(row['start_time'].strftime('%Y-%m-%d %H:00:00'), tz="Europe/Helsinki")
#     print(row['start_time'])
#     print(start_date_and_hour)
    start_date: str = pd.Timestamp(row['start_time'].strftime('%Y-%m-%d'))
    end_date_and_hour: str = pd.Timestamp(row['end_time'].strftime('%Y-%m-%d %H:00:00'), tz="Europe/Helsinki")
    chargeing_time: int = (row['end_time'] - row['start_time']).total_seconds()
    if chargeing_time > 0:
        energy_per_second = row['energy'] / chargeing_time
    else:
        energy_per_second = 0
    chargeing_time_list = pd.date_range(start_date_and_hour, end_date_and_hour, freq='H')
    mask: pd.Series = df_electricity_price['DateTime'].isin(chargeing_time_list)
    df_price_report: pd.DataFrame = df_electricity_price[mask]
    df_price_report['id'] = row['id']
    df_price_report['connector_id'] = row['connectorId']
    df_price_report['Autopaikka'] = row['Autopaikka']
    df_price_report['chargeing_time'] = 3600
    df_price_report['start_time'] = row['start_time']
    df_price_report['end_time'] = row['end_time']
    add_chargeing_time_column(df_price_report, row, start_date_and_hour, end_date_and_hour)
    df_price_report['kulutettu energia'] = df_price_report['chargeing_time'] * energy_per_second
    df_price_report['Lataus aika (min.)'] = np.round(df_price_report['chargeing_time'] / 60, 2)
    df_price_report['Kokonais summa + ALV'] = df_price_report['kulutettu energia'] * df_price_report['Summa+ALV']
    df_price_report["owner"] = set_owner(start_date, row['id'], row['connectorId'])
#     df_price_report.drop(['Price 0%', 'Price 10%', 'Price 24%'], axis=1, inplace=True)
    return df_price_report


In [ ]:
df_list = df_defa.apply(lambda row: create_list_of_chargeing_times(row, df_electricity_price), axis=1)
df_total_cost = df_list[0]
for df in df_list[1:]:
    df_total_cost = pd.concat([df_total_cost, df])
df_total_cost.reset_index(drop=True, inplace=True)


In [ ]:
df_cost: pd.DataFrame = df_total_cost[(df_total_cost["DateTime"] >= pd.to_datetime(start_date)) & \
            (df_total_cost["DateTime"] < (pd.to_datetime(end_date) + np.timedelta64(1,"D")))]


In [ ]:
df_cost.head()


In [ ]:
df_cost.tail()


In [ ]:
df_cost.shape


In [ ]:
# df_cost_groupby = df_cost.groupby(['id', 'connector_id', \
#                                    "owner", "Autopaikka"])
df_cost_groupby = df_cost.groupby(["Autopaikka", "owner"])


In [ ]:
chargeing_points:list = list(df_cost_groupby.groups.keys())


In [ ]:
df_bill_2022_06_15__2023_12_31: pd.DataFrame = \
                    pd.DataFrame(columns=["Autopaikka", "Omistaja", \
                          "Kulutus (kWh)", "Summa (€)"])


In [ ]:
i: int = -1
for group in df_cost_groupby:
    i += 1
    chargeing_point_name: str = group[0]
    df: pd.DataFrame = group[1]
    total_price: float = \
        df['Kokonais summa + ALV'].sum() / 100
    the_consumed_electricity: float = df['kulutettu energia'].sum()
    length: int = len(df_bill_2022_06_15__2023_12_31.index)
    df_bill_2022_06_15__2023_12_31.loc[length] = \
            {"Autopaikka": chargeing_point_name[0], \
             "Omistaja": chargeing_point_name[1],
             "Summa (€)": total_price,
             "Kulutus (kWh)": the_consumed_electricity}


In [ ]:
def create_sort_id(row):
    m = re.match("^\D+(\d+)$", row["Autopaikka"])
    sort_id: int = int(m.group(1))
    return sort_id



In [ ]:
df_bill_2022_06_15__2023_12_31["sort_id"] = df_bill_2022_06_15__2023_12_31.apply(lambda row: create_sort_id(row), axis=1)
df_bill_2022_06_15__2023_12_31.sort_values(["sort_id"], inplace=True)
df_bill_2022_06_15__2023_12_31.drop(["sort_id"], inplace=True, axis=1)
df_bill_2022_06_15__2023_12_31["Kulutus (kWh)"] = df_bill_2022_06_15__2023_12_31["Kulutus (kWh)"].round(2)
df_bill_2022_06_15__2023_12_31["Summa (€)"] = df_bill_2022_06_15__2023_12_31["Summa (€)"].round(2)
df_bill_2022_06_15__2023_12_31["Hinta/kWh(€)"] = (df_bill_2022_06_15__2023_12_31["Summa (€)"] / \
                                                  df_bill_2022_06_15__2023_12_31["Kulutus (kWh)"]).round(2)
print("Laskutus ajanjaksolta 15.06.2022 -- 31.12.2022 (Sisältäen aloitus ja lopetus päivät)\n")
print(df_bill_2022_06_15__2023_12_31[df_bill_2022_06_15__2023_12_31["Summa (€)"] > 0.0].to_string(index=False))
print("\nHuom: {}{}".format("Ennen aloituspäivää aloitettu lataus sisältää ainoastaan sen osan ajasta,",
                          " joka on aloituspäivän puolella."))
print("      {}{}".format("Lopetuspäivän jälkeen lopetettu lataus sisältää ainoastaan sen osan ajasta,",
                          "joka on lopetuspäivän puolella."))
print("\n      {}\n      {}".format(
                   "Laskutus perustuu tunti kohtaiseen sähkön hintaan. Jokaiselle tunnille on laskettu hinta,",
                   "joka koostuu sen hetkisestä Siirto, Vero, Marginaali ja Spot/Sopimus hintojen summasta kerrottuna ALV% (24%)."))
print("")
print("      E_<päiväys> --> {}".format("tarkoittaa omistajaa, jonka omistus on päättynyt kyseisenä päivänä."))
print("      S_<päiväys> --> {}".format("tarkoittaa omistajaa, jonka omistus on alkanut kyseisenä päivänä."))



In [ ]:
def create_file_path_for_chargeing_data(chargeing_point_name: tuple) -> str:
    if chargeing_point_name[1] == "--":
        file_path: str = "Output/{}.xlsx".format(chargeing_point_name[0])
    else:
        file_path: str = "Output/{}_{}.xlsx".format(chargeing_point_name[0], chargeing_point_name[1])
    return file_path


In [ ]:
chargeing_point_name = ('AK15', "--")

for group in df_cost_groupby:
    if chargeing_point_name == group[0]:
        df: pd.DataFrame = group[1]
        print("{}-{}".format(chargeing_point_name[0], chargeing_point_name[1]))
        file_path: str = create_file_path_for_chargeing_data(chargeing_point_name)
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            display(df[['DateTime', 'Energia', "Siirto", "Marginaali", "Vero", \
                        "kulutettu energia", 'Kokonais summa + ALV', 'Lataus aika (min.)']])
            df[['DateTime', 'Energia', "Siirto", "Marginaali", "Vero", \
                        "kulutettu energia", 'Kokonais summa + ALV', 'Lataus aika (min.)']].to_excel(file_path)
        break

In [ ]:
file_path: str = os.path.join(preprocessed_input_folder, mittalukemat_2023_01_01_file_name)
file = open(file_path, "r")
df_mittalukemat_2023_01_01: pd.DataFrame = pd.read_csv(file)
file.close()


In [ ]:
df_mittalukemat_2023_01_01.drop("Unnamed: 0", inplace=True, axis=1)
df_mittalukemat_2023_01_01.head(40)


In [ ]:
df_bill_2022_06_15__2023_12_31["Jo laskutettu (kWh)"] = 0.0


In [ ]:
for group in df_bill_2022_06_15__2023_12_31.groupby(["Autopaikka"]):
    autopaikka: str = group[0]
    df: pd.DataFrame = group[1]
    kulutus: float = df["Kulutus (kWh)"].sum()
    df_mittalukemat_2023_01_01.loc[df_mittalukemat_2023_01_01["Autopaikka"] == autopaikka, "Jo laskutettu (kWh)"] = kulutus


In [ ]:
print("Laskutus jossa kaikki ennen 15.06.2022 tehdyt lataukset\n")
df_mittalukemat_2023_01_01["Kulutus (kWh)"] = df_mittalukemat_2023_01_01["Mittarilukema"] - \
                                                df_mittalukemat_2023_01_01["Jo laskutettu (kWh)"]
df_mittalukemat_2023_01_01.drop(["DateTime", "Mittarilukema", "Jo laskutettu (kWh)"], axis=1, inplace=True)
df_mittalukemat_2023_01_01["Summa (€)"] = df_mittalukemat_2023_01_01["Kulutus (kWh)"] * 10.8686 / 100
df_mittalukemat_2023_01_01["Summa (€)"] = df_mittalukemat_2023_01_01["Summa (€)"].round(2)
df_mittalukemat_2023_01_01["Kulutus (kWh)"] = df_mittalukemat_2023_01_01["Kulutus (kWh)"].round(2)
print(df_mittalukemat_2023_01_01[df_mittalukemat_2023_01_01["Summa (€)"] > 0.1].to_string(index=False))
print("\nHuom: {}\n{}".format("Tämä laskutus on erillisenä koska ennen kesäkuuta 2022 tapahtuneista latauksista", \
                        "      ei ole saatavilla tarkempia lataus tietoja Defan cloudcharge järjestelmästä."))
print("\n      {}{}\n      {}".format("Laskutus tapahtuu kiinteällä 10.87 c€/kWh hinnalla,", \
                            " joka perustuu voimassa olleeseen kiinteään sopimukseen.",
                           "(toki siirto, vero ja ALV% mukana)."))


In [ ]:
description: str = """
Laskutus on toteutettu siten että jokaisella tunnilla on ajanmukainen Siirto, Vero, Marginaali ja Spot/Sopimus hinta.
kWh hinta on laskettu jokaiselle tunnille summaamalla kaikki hinnat yhteen ja lisätty ALV%, joka on taloyhtiölle 24%.

Laskutus ennen 21.10.2022 perustuu kiinteisiin sopimus hintoihin kulloisenkin ajankohdan mukaan.

21.10.2022 ja siitä eteenpäin taloyhtiössä on ollut käytössä spot hinta. Siitä lähtien jokaisella tunnilla on
sopimushinnan sijasta spot hinta, joka on otettu http://www.sahko.tk/ sivustolta.

Joulukuussa 2022 (12/2022) spot hinnan (ainoastaan spot hinnan) ALV10 on huomioitu. Multa osin ALV24 on käytetty.

Defan cloudcharge järjestelmästä ei saada tarkkaa tuntikohtaista kulutus tietoa.
Siitä johtuen laskutus perustuu seuraviin asioihin:
   1. Lataus aika alkaa siitä kun latauspistoke kytketään autoon ja päättyy kun lataus pistoke otetaan irti.
   2. Latausaikana ladattu sähkö (kWh) laskutetaan kyseisten tuntien mukaisella sähkön hinnalla.

Esimerkki:
    - Latauspistoke kytektty autoon klo 17:45
    - Latauspistoke poistettu autosta klo 20:15
    - Sähkön kulutus kyseisenä 2 tunnin ja 30 minuutin (=150 min) aikana on 9 kWh
    Yllä mainitut kolme tietoa saadaan Defan cloudcharge järjestelmästä

    Koska tarkkaa ajankohtaa lataukselle ei tiedetä lasketaan sähkön kulutus jokaiselle minuutille.
    Esimerkin tapauksessa sähköä on kulutettu (9 kwh / 150 min) 0.06 kwh/min.
    Tällöin tuntikohtainen sähkön kulutus on:
        17:00 --> 15 min * 0.06 kWh/min = 0.9 kWh
        18:00 --> 60 min * 0.06 kWh/min = 3.6 kWh
        19:00 --> 60 min * 0.06 kWh/min = 3.6 kWh
        20:00 --> 15 min * 0.06 kWh/min = 0.9 kWh

    Lopuksi jokaista tuntia laskutetaan kyseisen tunnin Spot hinnan mukaan
    (toki siirto, vero, marginaali ja ALV24 mukana).
    Huom: Joulukuussa 2022 spothinnan (vain spot hinnan) ALV on ollut 10%, muina aikaoina 24%.
"""

print(description)
